In [1]:
from HARK.distribution import Bernoulli, Lognormal, MeanOneLogNormal, Normal
from HARK.model import Control, DBlock, RBlock

from HARK.simulation.monte_carlo import MonteCarloSimulator

import math

In [2]:
calibration = {
    "DiscFac": 0.96,
    "CRRA": 2.0,
    "rho" : 0.95,
    "R": 1.03, # will get overridden
    "TranShkStd": 0.1,
    "debt_limit" : -1
}

In [3]:
consumption_block = DBlock(
    **{
        "name": "consumption",
        "shocks": {
            "theta": (Normal, {"mu" : 0, "sigma": "TranShkStd"}),
        },
        "dynamics": {
            "y": lambda y, rho, theta: y * rho + theta,
            "R" : Control(["a", "y"]),
            "w": lambda a, R, y: R * a + math.e ** y,
            "c": Control("w"), # constraint c < w - debt_limit
            "a": lambda w, c: w - c,
        },
        "reward": {
            "u": lambda c, CRRA: c ** (1 - CRRA) / (1 - CRRA),
            "d": lambda a, R: - a * R 
        },
    }
)

## How to set $R$

Let `R = Control(a, y)` be the rule for how money is lent.

This is a simple that avoids the state explosion of history.
But since `y` is autoregressive that's fine.

We can look at the dynamics with different rules.

### Todo: test with a couple variations of rules and plot the outcomes.

In [7]:
sim = MonteCarloSimulator(  ### Use fm, blockified
    calibration,
    consumption_block,
    {
        "R": lambda a, y : 1.03,
        "c": lambda w: w,
    },
    {  # initial states
        "a": 1,
        "y": 1.0,
    },
    agent_count=3,
    T_sim=120,
)

sim.initialize_sim()
hist = sim.simulate()

{'DiscFac': 0.96, 'CRRA': 2.0, 'rho': 0.95, 'R': array([nan, nan, nan]), 'TranShkStd': 0.1, 'debt_limit': -1, 'theta': array([ 0.01257302, -0.01321049,  0.06404227]), 'y': array([1., 1., 1.]), 'w': array([nan, nan, nan]), 'c': array([nan, nan, nan]), 'a': array([1., 1., 1.]), 'u': array([nan, nan, nan]), 'd': array([nan, nan, nan])}
{'DiscFac': 0.96, 'CRRA': 2.0, 'rho': 0.95, 'R': 1.03, 'TranShkStd': 0.1, 'debt_limit': -1, 'theta': array([ 0.01049001, -0.05356694,  0.03615951]), 'y': array([0.96257302, 0.93678951, 1.01404227]), 'w': array([3.64842508, 3.58177581, 3.78672192]), 'c': array([3.64842508, 3.58177581, 3.78672192]), 'a': array([0., 0., 0.]), 'u': array([-0.27409087, -0.27919112, -0.26408065]), 'd': array([-0., -0., -0.])}
{'DiscFac': 0.96, 'CRRA': 2.0, 'rho': 0.95, 'R': 1.03, 'TranShkStd': 0.1, 'debt_limit': -1, 'theta': array([ 0.1304    ,  0.0947081 , -0.07037352]), 'y': array([0.92493438, 0.8363831 , 0.99949966]), 'w': array([2.52170279, 2.30800404, 2.7169221 ]), 'c': arra

### Implement value backup on the two rewards, see if it reaches a steady state.

In [8]:
f = lambda a, b, c, d: a + b - c * d

In [11]:
args = {
    'a' : 1,
    'b': 2,
    'c' : 3,
    'd' : 4,
    'e' : 5
}

In [13]:
f(*args)

TypeError: <lambda>() takes 4 positional arguments but 5 were given